# 1D CME simulation to determine cell polarization via a signalling pathway
nicole pagane
2/15/17

### import necessary modules

In [20]:
%reset
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.integrate as snt
import scipy.stats as stats
from scipy.optimize import curve_fit
import os
import plotly.plotly as py
import plotly.graph_objs as go
from decimal import Decimal
from matplotlib.colors import LogNorm

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


### parameters

In [21]:
#radius of substrate (assume the kinase is of similar size)
r = 1e-9 #m

#one-dimensional length of lattice gradient
L = 1.0e-3 #m

#number of compartments **INCLUDING THE 2 CONSTANT BOUNDARY SUBVOLUMES AND 3 CENTRAL SUBVOLUMES OF THE CELL**
K = 9

#large number n for how many timesteps to use for plotting
n = 100

#sample space range for parameter scan of kinetic rate constants
s0 = 1e-5
s1 = 1e5
slen = 10

#original far left and far right compartment species numbers
N = {}
N['A'] = np.asarray([0, 20])

#initial number of kinase molecules
c2 = 100

#initial number of singly phosphoylated kinase molecules
c3 = 175

#initial number of doubly phosphorylated kinase molecules
c4 = 70

# filename for parameter trials
paramFile = "parameterMatrix.dat"
odeFile = "parameterTrial.dat"

#set to False for holistic (slow) parameter search or True for quick
quick = False

### setup for deterministic solution to cell signalling pathway 

In [22]:
## SPECIES ###
# S    substrate                        (c1)
# K    kinase                           (c2)
# Kp   phosphorylated kinase            (c3)
# Kpp  doubly phosphorlyated kinase     (c4)

### EQUATIONS FOR CELL SIGNALLING ###
# S + K -> S + Kp  (k1)
# K + K -> Kp + K  (k2)
# Kp+ K -> K + K   (k3)
# Kp+Kp -> Kpp+Kp  (k4)
# Kpp+Kp-> Kp+Kp   (k5)


nReac = 5
nSpec = 4

# define intital values and parameters for the above kinetic rate laws
# model the concentration of substrate as a linear gradient 
c1 = 0 # technically N['A'][0] is the correct initial concentration if looking at the boundary
       # but lets assume 0 for the next right cell 
V = 1e-15
y0 = np.asarray([c1, c2, c3, c4])/V #initial species concentration

params = {}
paramRange = np.linspace(s0, s1, slen)

#there should be paramRange^params columns so in our case len(paramRange)^nReac
if not quick:
    cols = len(paramRange)**nReac
else:
    cols = len(paramRange)**3

params['k1'] = np.zeros(cols)
params['k2'] = np.zeros(cols)
params['k3'] = np.zeros(cols)
params['k4'] = np.zeros(cols)
params['k5'] = np.zeros(cols)

keys = ['k1', 'k2', 'k3', 'k4', 'k5']

if quick:
    params['k1'] = np.ones(cols)*1
    params['k2'] = np.ones(cols)*1

ti = 0
tf = 10
t = np.linspace(ti, tf, n)

def rates(y, t, k1, k2, k3, k4, k5, S):
    return np.asarray([-k1*S*y[0] - k2*y[0]**2 + k3*y[1]*y[0],
                      k1*S*y[0] + k2*y[0]**2 - k3*y[1]*y[0] - k4*y[1]**2 + k5*y[2]*y[1],
                      k4*y[1]**2 -  k5*y[2]*y[1]])

### parameter scan setup

In [23]:
# check to see if there are parameter files to avoid having to run the next section
files = os.listdir(".")
if paramFile not in files:
    print("the parameter permutations have not been created"
          "\nthis may take a bit of time dependent on the given parameter range")
    runPerm = True
else:
    print('the parameter permutations have already been created')
    runPerm = False
    FID = open(paramFile, 'r')
    data = FID.readlines()
    for i in range(len(data)):
        data[i] = data[i].strip()
        data[i] = data[i].split()
        for j in range(len(keys)):
            params[keys[j]][i] = float(data[i][j])
    #close file now that data has been read in
    FID.close()
    print('parameter file read in')
        

the parameter permutations have already been created
parameter file read in


In [24]:
# the first parameter is the longest to set up, time improves for each successive parameter
# TIME IS ONLY AN ISSUE IF HOLISTIC SEARCH IS DONE
if runPerm:
    if not quick:
        iter = range(len(keys))
    else:
        iter = [0,1,2]
    for par in iter:
        indStart = 0
        while indStart < cols:
            val = 0
            while val < len(paramRange):
                indEnd = indStart + len(paramRange)**(par) + 1
                if (quick and par == 0):
                    par = 3 - 1 #index of k3
                elif (quick and par == 1):
                    par = 4 - 1 #index of k4
                elif (quick and par == 2):
                    par = 5 -1 #index of k5
                params[keys[par]][indStart:indEnd] = paramRange[val]
                val += 1
                indStart = indEnd 
                if (quick and par == 2):
                    par = 0 #return back to expected par value
                elif (quick and par == 3):
                    par = 1 #return back to expected par value
                elif (quick and par == 4):
                    par = 2 #return back to expected par value
        #print(float(par+1)/len(params.keys()))
    #save permutations file to avoid having to run the above code more than once
    print('writing parameters to file to avoid having to run again')
    FID = open(paramFile, 'w')
    for i in range(len(params['k1'])):
        line = [];
        for j in range(len(keys)):
            line.append(str(params[keys[j]][i]))
        FID.write(' '.join(line))
        FID.write('\n')
    #close file now that data has been read in
    FID.close()
    print('done')


In [37]:
tol = 1e-3
n = 50
Ktot = 1000
S = y0[0]
Kp = np.linspace(0,Ktot,n)
Kpp = np.linspace(0,Ktot,n)

diff = {}
acc = 0
for iter in range(len(params['k1'])):
    k1 = params['k1'][iter]
    k2 = params['k2'][iter]
    k3 = params['k3'][iter]
    k4 = params['k4'][iter]
    k5 = params['k4'][iter]
    #append to differential space
    A = np.matrix(np.asarray([
                -k1, -k2, k3, 0, 0,
                k1, k2, -k3, -k4, k5, 
                0, 0, 0, k4, -k5 ]).reshape([3,5]))
    #find individual kinase differentials
    dk = np.ones(len(Kp)*len(Kpp)).reshape([len(Kpp), len(Kp)])*1000
    dkp = np.ones(len(Kp)*len(Kpp)).reshape([len(Kpp), len(Kp)])*1000
    dkpp = np.ones(len(Kp)*len(Kpp)).reshape([len(Kpp), len(Kp)])*1000
    k = []
    for i in range(len(Kp)):
        for j in range(len(Kpp)):
            K = Ktot - Kp[i] - Kpp[j]
            if K >= 0:
                dk[i,j] = np.power(np.matmul(A,np.asarray([S*K,K**2,K*Kp[i],Kp[i]**2,Kp[i]*Kpp[j]]).reshape([5,1])),2)[0]
                dkp[i,j] = np.power(np.matmul(A,np.asarray([S*K,K**2,K*Kp[i],Kp[i]**2,Kp[i]*Kpp[j]]).reshape([5,1])),2)[1]
                dkpp[i,j] = np.power(np.matmul(A,np.asarray([S*K,K**2,K*Kp[i],Kp[i]**2,Kp[i]*Kpp[j]]).reshape([5,1])),2)[2]
    grad = dk + dkp + dkpp
    if (sum(sum(grad <= tol))>2):
        acc = acc + 1
        print sum(sum(grad<=tol)), 'at index', iter, 'of', len(params['k1'])
        diff[acc] = {}
        diff[acc]['dk'] = dk
        diff[acc]['dkp'] = dkp
        diff[acc]['dkpp'] = dkpp
    

3 at index 0 of 100000
3 at index 1 of 100000
3 at index 2 of 100000
3 at index 3 of 100000
3 at index 4 of 100000
3 at index 5 of 100000
3 at index 6 of 100000
3 at index 7 of 100000
3 at index 8 of 100000
3 at index 9 of 100000
3 at index 10 of 100000
3 at index 110 of 100000
3 at index 111 of 100000
3 at index 112 of 100000
3 at index 113 of 100000
3 at index 114 of 100000
3 at index 115 of 100000
3 at index 116 of 100000
3 at index 117 of 100000
3 at index 118 of 100000
3 at index 119 of 100000
3 at index 120 of 100000
3 at index 220 of 100000
3 at index 221 of 100000
3 at index 222 of 100000
3 at index 223 of 100000
3 at index 224 of 100000
3 at index 225 of 100000
3 at index 226 of 100000
3 at index 227 of 100000
3 at index 228 of 100000
3 at index 229 of 100000
3 at index 230 of 100000
3 at index 330 of 100000
3 at index 331 of 100000
3 at index 332 of 100000
3 at index 333 of 100000
3 at index 334 of 100000
3 at index 335 of 100000
3 at index 336 of 100000
3 at index 337 of 100

KeyboardInterrupt: 

In [41]:
#ok i dont really like this but this has to be very user heavy with necessary input
#generate length 100 arrays to look at the interactive plot with
trials = []
count = 0
#while count + 100 <= cols:
while count + 100 <= ind:
    trials.append([count, count + 100])
    count = count + 100
print len(trials) - 1
#print "k1 = ", params['k1'][0]
#print "k2 = ", params['k2'][0]


att = 100 # CHANGE THIS NUMBER HERE TO SEE THE PLOTS FOR A LIMITED SELECTION OF SOLUTIONS 

#plot at given att batch
fig = plt.figure(figsize=(20,20))
ind = 1
col = 1
for i in range(trials[att][0], trials[att][1]):
    ax = fig.add_subplot(col,add,ind)
    plt.imshow(diff[i], cmap = 'bwr', origin = 'lower')
    plt.xlabel('kpp(index)')
    plt.ylabel('kp (index)')
    plt.title('dk/dt')
    plt.colorbar()
    ind = ind + 1
    if ind % 10 and ind != 0:
        col = col + 1
    
plt.show()

NameError: name 'ind' is not defined

### test the parameter combinations to find acceptable range

In [ ]:
# create a dictionay to hold ODE values for each parameter combination trial
odeTrials = {}

# check to see if there is an ODE file to avoid having to run the next section
files = os.listdir(".")
if odeFile not in files:
    print("finding the numerical solutions to the system of ODEs")
    for i in range(len(params['k1'])):
        k1 = params['k1'][i]
        k2 = params['k2'][i]
        k3 = params['k3'][i]
        k4 = params['k4'][i]
        k5 = params['k4'][i]
        ode = snt.odeint(rates, y0[1:len(y0)],t, args = (k1, k2, k3, k4, k5, y0[0]))
        odeTrials[i] = ode
    print('done')
    print('writing the solutions to a file')
    FID = open(odeFile, 'w')
    for i in range(len(odeTrials.keys())):
        FID.write(' '.join([str(params['k1'][i]), str(params['k2'][i]), str(params['k3'][i]), 
                            str(params['k4'][i]), str(params['k5'][i])]))
        FID.write('\n')
        for j in range(len(odeTrials[i])):
            line = [];
            for k in range(nSpec):
                line.append(str(odeTrials[i][j][k]))
            FID.write(' '.join(line))
            FID.write('\n')
        FID.write('\n')
    #close file now that data has been read in
    FID.close()
    print('done')
else:
    print('the numerical solutions have been found\nreading in the file')
    FID = open(odeFile, 'r')
    data = FID.readlines()
    ind = 0;
    odeInd = 0;
    for i in range(len(params['k1'])):
        ind = ind + 1
        odeTrials[i] = np.asarray(np.zeros(n*nSpec).reshape([n, nSpec]))
        for j in range(n):
            data[ind] = data[ind].strip()
            data[ind] = data[ind].split()
            for k in range(nSpec):
                odeTrials[odeInd][j][k] = float(data[ind][k])
            ind = ind + 1
        odeInd = odeInd + 1
        ind = ind + 1
    #close file now that data has been read in
    FID.close()
    print('parameter file read in')

### plot kp by kpp with different differentials to represent the density

In [ ]:
fig = plt.figure(figsize=(10,8))

# kinetic rate constants
k1 = 1e-2
k2 = 5e-3
k3 = 1e-3
k4 = 5e-2
k5 = 5e-2

#equilibrium concentrations
S = 20
Ktot = 50
k = 1

Kp = np.linspace(0,50,100)
Kpp = np.linspace(0,50,100)

#append to differential space
A = np.matrix(np.asarray([
                -k1, -k2, k3, 0,
                k1, k2, -k3-k4, k5, 
                0, 0, k4, -k5 ]).reshape([3,4]))

grad = np.ones(len(Kp)*len(Kpp)).reshape([len(Kpp), len(Kp)])*100
for i in range(len(Kp)):
    for j in range(len(Kpp)):
        k = Ktot - Kp[i] - Kpp[j]
        if k >= 0:
            grad[i,j] = np.sum(np.power(A*np.asarray([S*k,k,Kp[i],Kpp[j]]).reshape([4,1]),2))

plt.contourf(grad, cmap = 'bwr')
plt.xlabel('kpp(index)')
plt.ylabel('kp (index)')
plt.title('kpp vs kp weighted by gradient magnitude')
plt.colorbar()

plt.show()

In [ ]:
Kp = np.linspace(0,100,100)
Kpp = np.linspace(0,100,100)

grad = np.matrix(np.ones(len(Kp)*len(Kpp)).reshape([len(Kpp), len(Kp)])*200)
for i in range(len(Kp)):
    for j in range(len(Kpp)):
        k = Ktot - Kp[i] - Kpp[j]
        if k >= 0:
            grad[i,j] = np.sum(np.power(A*np.asarray([S*k,k,Kp[i],Kpp[j]]).reshape([4,1]),2))


trace = go.Heatmap(z = grad.tolist())
data=[trace]
py.iplot(data, filename='basic-heatmap')


### plot dkpp/dt for the numerical solutions of the kinetic laws to determine best fit parameters

In [ ]:
cols = sum(grad < 1) 
mins = [];
for i in cols.tolist()[0]:
    if i > 0:
        for j in range(len(grad[i,:])):
            if grad[i,j] < 1:
                mins.append(grad[i.j])
                
print mins

In [ ]:
odeTrials = {}
k1 = 2.8735e-17
k2 = .7208
k3 = .0288
k4 = .4796
k5 = .4996
ind = 0
for i in range(175,200):
    for j in range(75,100):
        for k in range(20):
            y0 = [0, i, j, k]
            ode = snt.odeint(rates, y0[1:len(y0)],t, args = (k1, k2, k3, k4, k5, y0[[0]]))
            odeTrials[ind] = []
            odeTrials[ind] = ode
            ind = ind+ 1
            
        

In [ ]:
#ok i dont really like this but this has to be very user heavy with necessary input
#generate length 100 arrays to look at the interactive plot with
trials = []
count = 0
#while count + 100 <= cols:
while count + 100 <= ind:
    trials.append([count, count + 100])
    count = count + 100
print len(trials) - 1
#print "k1 = ", params['k1'][0]
#print "k2 = ", params['k2'][0]


att = 100 # CHANGE THIS NUMBER HERE TO SEE THE PLOTS FOR A LIMITED SELECTION OF SOLUTIONS 

#create traces
data = []
for i in range(trials[att][0], trials[att][1]):
    dkpp = np.diff(odeTrials[i][:,3])/np.diff(t)
    trace = go.Scatter(
    x = odeTrials[i][:,3][0:n-1],
    y = dkpp,
    mode = 'lines',
    #name = ''.join([str('%.2E' % Decimal(params['k3'][i])), ", ", str('%.2E' % Decimal(params['k4'][i],2)), 
     #              ", ", str('%.2E' % Decimal(params['k5'][i],2))])
    )
    data.append(trace)

py.iplot(data, filename='scatter-mode')

In [ ]:
# define rates equations
def rates(y, t, k1, k2, k3, k4, k5):
    return np.asarray([c1 + (N['A'][1] - N['A'][0])*(L/(K-2))/L, 
                       -k1*y[0]*y[1] - k2*y[1] + k3*y[2],
                       k1*y[0]*y[1] + k2*y[1] - k3*y[2] - k4*y[2] + k5*y[3],
                       k4*y[2] -  k5*y[3]])

# plot species
k1 = .7038
k2 = 2.8102e-14
k3 = 0.0683
k4 = .5
k5 = .5

#initial concentrations
c1 = 0
c2 = 20
c3 = 10
c4 = 5
y0 = np.asarray([c1, c2, c3, c4])/V #initial species concentration


ode = snt.odeint(rates, y0,t, args = (k1, k2, k3, k4, k5))

#find dkpp/dt
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,2,1)
dkpp = np.diff(ode[:,3])/np.diff(t)
ax.plot(ode[:,3][0:n-1],dkpp)
ax.set_title('bistable solution with intesections at 0')
ax.set_xlabel('[kpp]')
ax.set_ylabel('dkpp/dt')

#plot two parameters against each other to determine bistable solution
ax = fig.add_subplot(1,2,2)
ax.plot(k2, k5, 'o')


fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,2,1)
for i in range(len(y0)):
    ax.plot(t, ode[:,i][0:n])
ax.set_xlabel('t (s)')
#plt.title(''.join(['[S] = ', str(c1)]))
ax.set_ylabel('species concentration, mM')
ax.legend(['S', 'K', 'Kp', 'Kpp'])

#zoom in 
ax = fig.add_subplot(1,2,2)
tf = 10
t = np.linspace(ti, tf, n)
ode = snt.odeint(rates, y0,t, args = (k1, k2, k3, k4, k5))
for i in range(len(y0)):
    ax.plot(t, ode[:,i][0:n])
ax.set_xlabel('t (s)')
ax.set_title('zoomed in to see species fluctuations early on')
ax.set_ylabel('species concentration')
ax.legend(['S', 'K', 'Kp', 'Kpp'])
ax.set_xlim([0,1])


plt.show()

In [ ]:
45.5/46.5

In [ ]:
#plot fration of occupied phosphorylation sites to determine of parameters yield cooperativity 
# (sigmoidal as opposed to hyperbolic)
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,2,1)
ax.plot(ode[:,0], (ode[:,2] + 2*ode[:,3])/(2*(ode[:,1] + ode[:,2] + ode[:,3])))
ax.set_ylabel('fration of occupied phosphorylation sites')
ax.set_xlabel('[S]')
ax = fig.add_subplot(1,2,2)
ax.plot(ode[:,0], (ode[:,2] + 2*ode[:,3])/(2*(ode[:,1] + ode[:,2] + ode[:,3])))
ax.set_ylabel('fration of occupied phosphorylation sites')
ax.set_xlabel('[S]')
plt.title('zoomed in')
ax.set_xlim([0, 20])

plt.show()

In [ ]:
# Hill equation
def Hill(x, n , kD):
    return (x**n)/(kD + x**n)

popt, pcov = curve_fit(Hill, ode[:,0],(ode[:,2] + 2*ode[:,3])/(2*(ode[:,1] + ode[:,2] + ode[:,3])))
plt.plot(ode[:,0],(ode[:,2] + 2*ode[:,3])/(2*(ode[:,1] + ode[:,2] + ode[:,3])), 'o', label='Data')
plt.plot(ode[:,0],Hill(ode[:,0], popt[0], popt[1]), 'r-', label='Fit')
plt.legend()
print(''.join(['Hill constant, n: ', str(popt[0]), '\n', 'Dissociation constant, kD: ', str(popt[1])]))
plt.show()

### same deterministic solution as above but now with new definition for autophosphorylation reactions
 new definition forces nonlinearity 

In [ ]:
## SPECIES ###
# S    substrate                        (c1)
# K    kinase                           (c2)
# Kp   phosphorylated kinase            (c3)
# Kpp  doubly phosphorlyated kinase     (c4)

### EQUATIONS FOR CELL SIGNALLING ###
# S + K -> S + Kp  (k1)
# K + K -> Kp + K  (k2)
# Kp+ K -> K + K   (k3)
# Kp+Kp -> Kpp+Kp  (k4)
# Kpp+Kp-> Kp+Kp   (k5)

# plot species
k1 = .7038
k2 = 2.8102e-14
k3 = 0.0683
k4 = .5
k5 = .5

#initial concentrations
V = 1
c1 = 20
c2 = 10
c3 = 5
S = 20
y0 = np.asarray([c1, c2, c3])/V #initial species concentration
t = np.linspace(0,10,100)

# define rates equations
def rates(y, t, k1, k2, k3, k4, k5, S):
    return np.asarray([-k1*S*y[0] - k2*y[0]**2 + k3*y[1]*y[0],
                      k1*S*y[0] + k2*y[0]**2 - k3*y[1]*y[0] - k4*y[1]**2 + k5*y[2]*y[1],
                      k4*y[1]**2 -  k5*y[2]*y[1]])
# solve the ODEs
ode = snt.odeint(rates, y0,t, args = (k1, k2, k3, k4, k5, S))

#find dkpp/dt
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,2,1)
dkpp = np.diff(ode[:,2])/np.diff(t)
ax.plot(ode[:,2][0:n-1],dkpp)
ax.set_title('bistable solution with intesections at 0')
ax.set_xlabel('[kpp]')
ax.set_ylabel('dkpp/dt')

#plot two parameters against each other to determine bistable solution
ax = fig.add_subplot(1,2,2)
ax.plot(k2, k5, 'o')


fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,2,1)
for i in range(len(y0)):
    ax.plot(t, ode[:,i][0:n])
ax.set_xlabel('t (s)')
#plt.title(''.join(['[S] = ', str(c1)]))
ax.set_ylabel('species concentration, mM')
ax.legend(['S', 'K', 'Kp', 'Kpp'])

#zoom in 
ax = fig.add_subplot(1,2,2)
tf = 10
ode = snt.odeint(rates, y0,t, args = (k1, k2, k3, k4, k5, S))
for i in range(len(y0)):
    ax.plot(t, ode[:,i][0:n])
ax.set_xlabel('t (s)')
ax.set_title('zoomed in to see species fluctuations early on')
ax.set_ylabel('species concentration')
ax.legend(['S', 'K', 'Kp', 'Kpp'])
ax.set_xlim([0,1])


plt.show()